## Project Description

This project is a regression project that aims to predict the IQ of a baby based on a set of input features from their mother. The stages involved in this project are as follows:

1. Data Exploration: In this stage, we analyze the dataset to gain insights into its structure, identify potential relationships between variables, and understand the distribution of the target variable.

2. Data Cleaning: In this stage, we handle missing values, outliers, and any inconsistencies in the dataset. This involves techniques such as imputation, removal of outliers, and data normalization.

3. Data Analysis: In this stage, we perform exploratory data analysis (EDA) to further understand the relationships between variables, identify patterns, and detect any potential issues that may affect the regression model.

4. Model Development: In this stage, we develop a regression model using appropriate algorithms such as linear regression, decision trees, or ensemble methods. We train the model on the cleaned dataset and tune its hyperparameters to optimize its performance.

5. Model Testing: In this stage, we evaluate the performance of the regression model using appropriate evaluation metrics such as mean squared error (MSE), root mean squared error (RMSE), or R-squared. We assess the model's ability to generalize to unseen data and make predictions on new instances.

Throughout these stages, we may iterate and refine our approach based on the insights gained and the performance of the model. The ultimate goal is to build an accurate regression model that can effectively predict the target variable based on the given input features.


In [3]:
import pandas as pd
import sys

# 1. Data exploration

In [9]:
sys.path.append('../DATA/iq_data')
data_child = pd.read_csv('../DATA/iq_data/child_iq.csv')
data_kid = pd.read_csv('../DATA/iq_data/kid_iq.csv')

data_child -> provides information like the child' test score at age 3 (ppvt), the mother's age at childbirth (momage), and the mother's education level (educ_cat, where 1 stands for no HS education, 2 = HS grad, 3 = some college, and 4 = college grad).

data_kid -> provides information on the kid's score (kid_score), if the mother went to high school or not (mom_hs), on the mother's IQ (mom_iq), on her job (mom_work), and finally on her age (mom_age)